# Map

In [1]:
import ast
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from geolocation.main import GoogleMaps
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

from googlemaps_key import KEY

## Autores

In [6]:
%%time

autores_df = pd.read_csv(
    'autores_colombianos_4.csv',
    header=0,
    converters={
        'GEOLOC_CAPITAL_AUTOR': ast.literal_eval,
        'GEOLOC_CAPITAL_EJEMPLAR': ast.literal_eval,
        'GEOLOC_CIUDAD_AUTOR': ast.literal_eval
    }
)
autores_df = autores_df.fillna('')

CPU times: user 369 ms, sys: 5.85 ms, total: 375 ms
Wall time: 374 ms


In [9]:
autores_df[autores_df.CIUDAD_AUTOR == 'San Andrés'].drop_duplicates(['AUTOR'])

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO,NOMBRE_ENCONTRADO,GEOLOC_CIUDAD_AUTOR
146,Antonio Nariño : el colombiano de todos los ti...,Colombia,Antonio Cacua Prada,Colombia,2012,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",San Andrés,Santander,True,Antonio Cacua Prada,"(6.811888, -72.848242)"


In [17]:
autores_df.shape

(5551, 12)

In [18]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB,CAPITAL_AUTOR,CAPITAL_EJEMPLAR,GEOLOC_CAPITAL_AUTOR,GEOLOC_CAPITAL_EJEMPLAR,CIUDAD_AUTOR,DEPARTAMENTO_AUTOR,ENCONTRADO
0,En la laguna más profunda,Colombia,Oscar Collazos,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",Bahía Solano,Chocó Department,True
1,El gato bandido y otros cuentos,Colombia,Rafael Pombo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
2,Cuentos pintados,Colombia,Rafael Pombo,Colombia,2011,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
3,Aleida : 10 años,Colombia,Vladdo,Colombia,2007,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False
4,Mitos y leyendas indígenas de Colombia,Colombia,Javier Ocampo López,Colombia,2013,Bogotá,Bogotá,"(4.710988599999999, -74.072092)","(4.710988599999999, -74.072092)",DESCONOCIDO,DESCONOCIDO,False


In [19]:
with open('autores_bogotanos_test.txt') as f:
    for line in f:
        a = line.strip()
        if a in autores_df['AUTOR'].unique():
            print(a)

Arturo Abella Rodríguez
Germán Arciniegas
Eduardo Caballero Calderón
Celso Román
Alfonso Dávila Ortiz
Santiago Gamboa
Alfredo Iriarte
Fernando Molano Vargas
Rafael Pombo
Daniel Samper Pizano
Mauricio Silva Guzmán
Juan Gabriel Vásquez
Alberto Zalamea
Jorge Zalamea


In [20]:
with open('autores_bogotanos_test.txt') as f:
    for line in f:
        a = line.strip()
        if a in autores_df['AUTOR'].unique():
            print(autores_df[autores_df['AUTOR'] == a])

                                                 TITULO PAIS_EJEMPLAR  \
121                      Don dinero en la independencia      Colombia   
702                              El florero de Llorente      Colombia   
896     Faldas, pero también sotanas en la regeneración      Colombia   
1326                                     Laureano Gómez      Colombia   
1572                                              Nuñez      Colombia   
1793          Sangre y misterios de la Plaza de Bolívar      Colombia   
2836  Grandes metidas de pata en la historia de Colo...      Colombia   
2863                             El florero de Llorente      Colombia   
2937                              Así fué el 9 de abril      Colombia   
3043                     Don dinero en la independencia      Colombia   
3128                             Así fué el 13 de junio      Colombia   
3300                             El florero de Llorente      Colombia   
3301                             El florero de Llor

In [15]:
autores_df['CIUDAD_AUTOR'].value_counts()

DESCONOCIDO             2182
Medellín                 432
Aracataca                373
Cartagena de Indias      253
Barranquilla             179
Cali                     146
Socha                    102
Popayán                   98
Bucaramanga               89
Jericó                    72
Herveo                    64
Zipaquirá                 58
Santo Domingo             56
Rivera                    52
Santa Cruz de Lorica      49
Bahía Solano              49
San Andrés                42
La Tebaida, Quindío       41
Honda                     41
Chaparral                 40
Filandia                  39
Amalfi                    36
San Juan de Pasto         35
Chiquinquirá              31
Donmatías                 31
Cúcuta                    30
Villa del Rosario         30
Abejorral                 29
Palmira                   28
Tolú                      28
                        ... 
Baraya                     1
Corinto, Cauca             1
Puerto Tejada              1
El Banco      

## Graph

### Rutas

In [5]:
rutas_df = autores_df[autores_df['PAIS_EJEMPLAR'] != 'Colombia']

In [6]:
len(rutas_df)

489

In [7]:
rutas_graph = nx.Graph()

for i, x in rutas_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not rutas_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        rutas_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        rutas_graph.node[ciudad]['weight'] += 1

In [12]:
rutas_graph.number_of_nodes()

61

### Ciclos

In [8]:
ciclos_df = autores_df[autores_df['PAIS_EJEMPLAR'] == 'Colombia']

In [9]:
len(ciclos_df)

2880

In [10]:
ciclos_graph = nx.Graph()

for i, x in ciclos_df.iterrows():
    ciudad = x['CIUDAD_AUTOR']
    if not ciclos_graph.has_node(ciudad):
        departamento = x['DEPARTAMENTO_AUTOR']
        geoloc = x['GEOLOC_CIUDAD_AUTOR']
        ciclos_graph.add_node(
            ciudad,
            departamento=departamento,
            latitude=geoloc[0],
            longitude=geoloc[1],
            weight=1
        )
    else:
        ciclos_graph.node[ciudad]['weight'] += 1

In [13]:
ciclos_graph.number_of_nodes()

138

# Saving results

In [11]:
nx.write_gexf(ciclos_graph, 'colombianos_ciclos_graph.gexf')
nx.write_gexf(rutas_graph, 'colombianos_rutas_graph.gexf')